# Klassifikation von Rot- und Weißwein mit Keras
In diesem Notebook führen wir die vollständige Datenaufbereitung, Modellierung, Visualisierung und Evaluierung eines neuronalen Netzes zur Unterscheidung von Rot- und Weißwein durch.

In [2]:
import pandas as pd
import numpy as np
import keras as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2025-05-10 14:24:46.061497: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 14:24:47.017438: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 14:24:47.505309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746879888.000848  133015 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746879888.302185  133015 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746879889.574026  133015 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

## Einlesen der Weindaten
Die CSV-Dateien befinden sich im Unterordner `data/processed`. Sie enthalten standardisierte chemische Merkmale für Rot- und Weißweine.

In [3]:
white = pd.read_csv('../data/processed/winequality-white.csv', sep=';')
red = pd.read_csv('../data/processed/winequality-red.csv', sep=';')

## Labeln und Zusammenführen der Datensätze
Für supervised learning bönitgen wir labels:
- Rotwein erhält das Label `1`
- Weißwein das Label `0`

Danach werden die Daten zusammengeführt.

In [4]:
red['label'] = 1
white['label'] = 0
wines = pd.concat([red, white], ignore_index=True)

## Feature-Matrix und Zielvariable erzeugen
Die ersten 11 Spalten (chemische Eigenschaften) dienen als Input-Variablen
(Features). Die Zielvariable `y` ist das binäre Label (1 = Rotwein, 0 = Weißwein),
und wird in 1D-Array umgewandelt.

In [5]:
x = wines.iloc[:, 0:11]
y = np.ravel(wines['label'])

## Aufteilung in Trainings- und Testdaten
Der Datensatz wird im Verhältnis 70:30 in Trainings- und Testdaten aufgeteilt
(70% Trainingsdaten).

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.3, random_state = 42)

## Skalierung der Merkmale
Die Daten werden mit dem `StandardScaler` auf Mittelwert 0 und Standardabweichung
 1 standardisiert. Unterschiedliche Skalen der Inputvariablen (z. B. pH vs.
 Alkohol) können das Training stören. Der Fit erfolgt
 **nur auf dem Trainingsset, aber: Testdaten müssen auch transformiert werden**.

In [7]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

## Klassifikation von Weinen (rot vs. weiß) als binäres Klassifikationsproblem

Ziel: Vorhersage d. binären Labels (0 = rot, 1 = weiß) anhand chemischer
Eigenschaften

Das neuronale Netz wird an diese Problemstruktur angepasst:
- ein einzelnes Ausgabeneuron mit Sigmoid-Aktivierung (für binäre Klassifikation)
- Eingabedimension entspricht der Anzahl der Input-Features (hier: 11)


## Aufbau des neuronalen Netzes

Wir definieren ein sequentielles Keras-Modell, , in dem Layer nacheinander
hinzugefügt werden. Hyperparameter wie Anzahl der Hidden-Layer, Neuronenanzahl,
Aktivierungsfunktionen etc. sind frei wählbar. Die Sturktur ist wie folgt:
- Eingabeschicht: 12 Neuronen, ReLU, `input_dim=11`
- Hidden Layer: 8 Neuronen, ReLU
- Ausgabeschicht: 1 Neuron, Sigmoid (für binäre Klassifikation)

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Dense(units=12, activation='relu', input_dim=11))
model.add(K.layers.Dense(units=8, activation='relu'))
model.add(K.layers.Dense(units=1, activation='sigmoid'))

## Weitere Hintergründe

1. Erstellen eines sequenziellen Keras-Modells
2. Erste Schicht (Input-Layer + erste Dense-Schicht)
    + 12 Neuronen als Startwert (guter Richtwert, entspricht etwa der Feature-Anzahl)
    + Aktivierungsfunktion: ReLU (Standard bei Hidden-Layern)
    + input_dim: 11, da 11 Input-Features (chemische Eigenschaften)
3. Zweite Schicht (Hidden-Layer)
    + 8 Neuronen (leichte Reduktion gegenüber erster Schicht)
    + erneut ReLU-Aktivierung
4. Ausgabeschicht für binäre Klassifikation
    + 1 Neuron (0 = rot, 1 = weiß)
    + Aktivierungsfunktion: Sigmoid (liefert Werte zwischen 0 und 1 geeignet für
     binäre Klassen)
5. Für mehrklassige Klassifikation würde man stattdessen:
    + mehrere Ausgabeneuronen (entsprechend der Klassenzahl)
    + und eine 'softmax'-Aktivierung verwenden

## Kompilierung des Modells

Festlegen von Optimierer, Verlustfunktion und Metriken

- Optimierer: `Adam` (state-of-the-art für viele Probleme, adaptiv)
- Verlustfunktion: `binary_crossentropy` (geeignet für binäre Klassifikation)
- Metrik: `accuracy` (Anteil korrekt klassifizierter Beispiele pro Epoche)

In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## Modelltraining
Wir trainieren das Modell 20 Epochen lang mit `validation_split=0.3`.

In [ ]:
hist = model.fit(x_train, y_train, epochs = 20, validation_split = 0.3)

- `x_train`, `y_train` sind die vorbereiteten Trainingsdaten
- `epochs = 20`: 20 Trainingsdurchläufe (kann bei Bedarf erhöht werden)
- `validation_split = 0.3`: 30% der Trainingsdaten werden zur Validierung abgezweigt
- **Achtung:** dies betrifft nur `x_train`/`y_train`, nicht die vorab separat
gehaltenen Testdaten!

## Verlauf der Genauigkeit während des Trainings
Die Trainings- und Validierungsgenauigkeit werden für jede Epoche geplottet.

In [ ]:
plt.plot(hist.history['accuracy'], label='Train Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoche')
plt.ylabel('Genauigkeit')
plt.legend()
plt.tight_layout()
plt.savefig('../figs/training_accuracy.png')
plt.show()

## Evaluation auf Testdaten
Nach dem Training evaluieren wir das Modell auf der echten Testmenge und geben die Loss- und Accuracy-Werte aus.

In [12]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Testverlust: {test_loss:.4f}")
print(f"Testgenauigkeit: {test_acc:.4f}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9948 - loss: 0.0260
Testverlust: 0.0342
Testgenauigkeit: 0.9928


**Hinweis:**
- Die Validierung während des Trainings basiert auf `validation_split`
- Die finale Testgenauigkeit stammt aus einer separaten, vorher unberührten Testmenge
- Dadurch erhalten wir eine realistische Einschätzung der Generalisierungsfähigkeit